In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle
from helper_functions import reduce_mem_usage
import numpy as np 
import pandas as pd 
from datetime import datetime as dt
import os
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
import lightgbm as lgb
import xgboost as xgb
import time
import datetime
from tqdm import tqdm_notebook as tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn import metrics
import gc
from tqdm import tqdm_notebook
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls


In [2]:
dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float32',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float32',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float32',
        'AVProductsEnabled':                                    'float32',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float32',
        'GeoNameIdentifier':                                    'float32',
        'LocaleEnglishNameIdentifier':                          'int8',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float32',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float32',
        'IeVerIdentifier':                                      'float32',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float32',
        'UacLuaenable':                                         'float32',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float32',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float32',
        'Census_ProcessorManufacturerIdentifier':               'float32',
        'Census_ProcessorModelIdentifier':                      'float32',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float32',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float32',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float32',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float32',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float32',
        'Census_IsFlightsDisabled':                             'float32',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float32',
        'Census_FirmwareManufacturerIdentifier':                'float32',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float32',
        'Census_IsVirtualDevice':                               'float32',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float32',
        'Wdft_IsGamer':                                         'float32',
        'Wdft_RegionIdentifier':                                'float32',
        'HasDetections':                                        'int8'
        }

In [3]:
train = pd.read_csv(r'C:\Users\jithi\OneDrive\Desktop\ML project\microsoft-malware-prediction\train.csv',dtype=dtypes)

In [4]:
#for each column print data type and 2 sample values
for col in train.columns:
    print(f"{col}: {train[col].dtype}, {train[col].sample(2).values}")

MachineIdentifier: category, ['7ac5958f30ae9aa155d829b3698b677c', 'f2f678206e129fafa51f37f0a0271195']
Categories (8921483, object): ['0000028988387b115f69f31a3bf04f09', '000007535c3f730efa9ea0b7ef1bd645', '000007905a28d863f6d0d597892cd692', '00000b11598a75ea8ba1beea8459149f', ..., 'ffffef606490b2970873ec0a27ebd24b', 'fffff8a40070d2d8379bb7fa2ed2fa66', 'fffffbbaaf5969ae4b93e7f3f6d7132f', 'ffffff75ba4f33d938ccfdb148b8ea16']
ProductName: category, ['win8defender', 'win8defender']
Categories (6, object): ['mse', 'win8defender', 'mseprerelease', 'windowsintune', 'fep', 'scep']
EngineVersion: category, ['1.1.15100.1', '1.1.15200.1']
Categories (70, object): ['1.1.12805.0', '1.1.12902.0', '1.1.13000.0', '1.1.13103.0', ..., '1.1.11202.0', '1.1.11104.0', '1.1.12802.0', '1.1.10701.0']
AppVersion: category, ['4.18.1807.18075', '4.14.17639.18041']
Categories (110, object): ['4.10.14393.0', '4.10.14393.1066', '4.10.14393.1198', '4.10.14393.1593', ..., '4.13.17618.1000', '4.12.16299.0', '4.8.10240.1

In [ ]:
numerics = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_columns = [c for c,v in dtypes.items() if v in numerics]
categorical_columns = [c for c,v in dtypes.items() if v not in numerics]

train_y = train['HasDetections']

train = reduce_mem_usage(train)

Mem. usage decreased to 2300.96 Mb (16.6% reduction)


In [ ]:
toDrop=['IsBeta','AutoSampleOptIn','Census_DeviceFamily',
        'Census_ProcessorManufacturerIdentifier','ProductName',
        'Census_IsPortableOperatingSystem','OsVer','Census_IsWIMBootEnabled',
        'Platform', 'PuaMode', 'Census_IsPenCapable', 'Census_IsFlightsDisabled',
        'Census_ProcessorClass', 'Census_IsFlightingInternal', 'Census_InternalBatteryType',
        'Census_IsAlwaysOnAlwaysConnectedCapable', 'UacLuaenable', 'Census_OSBuildNumber',
        'Census_OSArchitecture', 'Census_IsTouchEnabled', 'Census_ThresholdOptIn', 'Census_ThresholdOptIn', 'SMode',
        'HasTpm'
       ]
train.drop(toDrop, axis=1, inplace=True)

def rename_edition(x):
    x = x.lower()
    if 'core' in x:
        return 'Core'
    elif 'pro' in x:
        return 'pro'
    elif 'enterprise' in x:
        return 'Enterprise'
    elif 'server' in x:
        return 'Server'
    elif 'home' in x:
        return 'Home'
    elif 'education' in x:
        return 'Education'
    elif 'cloud' in x:
        return 'Cloud'
    else:
        return x
    
def encode_categorical_columns(x_train, columns, to_drop=None, sort=True):
    from tqdm import tqdm
    import pandas as pd

    to_drop = to_drop if to_drop is not None else []

    print("Training length:", x_train.shape[0])

    for col in tqdm(columns):
        if col in to_drop or col in ['MachineIdentifier', 'HasDetections']:
            continue

        # Factorize only on train data
        x_train[col], uniques = pd.factorize(x_train[col], sort=sort)
        x_train[col] = x_train[col].astype('int32')

        # Create a mapping dictionary from unique values
        category_map = {cat: idx for idx, cat in enumerate(uniques)}

        # Map test data using the train-derived mapping

        # Optional: replace missing (unseen categories in test) with -1 or 0
        print(f"{col} encoded. Train shape: {x_train.shape}")

    return x_train,category_map
def encode_categorical_columns_test(x_test, columns, category_map):
    from tqdm import tqdm
    import pandas as pd

    for col in tqdm(columns):
        if col in ['MachineIdentifier']:
            continue

        # Map test data using the train-derived mapping
        x_test[col] = x_test[col].map(category_map)
    return x_test


one_hot = {}
def add_features(df):
    global one_hot
    df['primary_drive_c_ratio'] = df['Census_SystemVolumeTotalCapacity']/ df['Census_PrimaryDiskTotalCapacity'];
    df['primary_drive_c_ratio'] = df['primary_drive_c_ratio'].astype('float32')
    
    df['non_primary_drive_MB'] = df['Census_PrimaryDiskTotalCapacity'] - df['Census_SystemVolumeTotalCapacity'];
    df['non_primary_drive_MB'] = df['non_primary_drive_MB'].astype('float32')
    
    df['aspect_ratio'] = df['Census_InternalPrimaryDisplayResolutionHorizontal']/ df['Census_InternalPrimaryDisplayResolutionVertical'];
    df['aspect_ratio'] = df['aspect_ratio'].astype('float16')

    df['dpi'] = ((df['Census_InternalPrimaryDisplayResolutionHorizontal'].astype('float32')**2 + df['Census_InternalPrimaryDisplayResolutionVertical'].astype('float32')**2)**.5)/(df['Census_InternalPrimaryDiagonalDisplaySizeInInches'].astype('float32'));
    df['dpi']=df['dpi'].astype('float32')

    df['Screen_Area'] = (df['aspect_ratio']* (df['Census_InternalPrimaryDiagonalDisplaySizeInInches']**2))/(df['aspect_ratio']**2 + 1);
    df['Screen_Area'] = df['Screen_Area'].astype('float32')

    df['AVProductsInstalled_slim']= df['AVProductsInstalled']
    df.loc[df['AVProductsInstalled'].isin([1, 2]) == False, 'AVProductsInstalled_slim'] = 3


    top_15 = df['AVProductStatesIdentifier'].value_counts(dropna=False, normalize=True).cumsum().index[:15]
    df['magic_4'] = 0
    df.loc[df['AVProductStatesIdentifier'].isin(top_15) == True, 'magic_4'] = 1
    
    df['Census_OSEdition'] = df['Census_OSEdition'].astype(str)
    df['Census_OSEdition'] = df['Census_OSEdition'].apply(rename_edition)
    df['Census_OSEdition'] = df['Census_OSEdition'].astype('category')

    return df

In [ ]:
train['SmartScreen'].replace({ '0':'off', '00000000':'off','BLOCK':'Block', 'off':'Off', 
                              'on':'On', 'requireadmin' : 'RequireAdmin', 'Enabled' : 'on', 
                              'OFF' : 'Off', 'Promt' : 'Prompt', 'prompt' : 'Prompt', 'on' : 'On', 
                              'off' : 'Off', 'warn' : 'RequireAdmin', 'requireAdmin' : 'RequireAdmin', '&#x03;' : '&#x01;'}, inplace=True)


train.head()
train=add_features(train)
train,category_map= encode_categorical_columns(train,  categorical_columns,to_drop=toDrop)
train = reduce_mem_usage(train)
with open('category_map.pkl', 'wb') as f:
    pickle.dump(category_map, f)


C:\Users\jithi\AppData\Local\Temp\ipykernel_540\3393876572.py:1: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



C:\Users\jithi\AppData\Local\Temp\ipykernel_540\3393876572.py:1: FutureWarning:

The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.



Training length: 8921483


 10%|█         | 3/30 [00:00<00:01, 20.98it/s]

EngineVersion encoded. Train shape: (8921483, 67)
AppVersion encoded. Train shape: (8921483, 67)


 20%|██        | 6/30 [00:00<00:01, 12.32it/s]

...

 97%|█████████▋| 29/30 [00:02<00:00,  8.22it/s]

Census_ActivationChannel encoded. Train shape: (8921483, 67)


100%|██████████| 30/30 [00:03<00:00, 10.00it/s]

Census_FlightRing encoded. Train shape: (8921483, 67)


Mem. usage decreased to 2189.36 Mb (20.7% reduction)


In [ ]:
#save train and test as kaggle_encoded_csv
train.to_csv('kaggle_encoded_train.csv', index=False)


# Saving a scaled version without NAN's for Neural Networks


In [ ]:
#kaggle encodings
import pandas as pd
import numpy as np

dtypes = {
    'MachineIdentifier': 'category',
    'EngineVersion': 'float64',
    'AppVersion': 'float64',
    'AvSigVersion': 'float64',
    'RtpStateBitfield': np.float16,
    'IsSxsPassiveMode': np.int8,
    'DefaultBrowsersIdentifier': np.float16,
    'AVProductStatesIdentifier': np.float32,
    'AVProductsInstalled': np.float16,
    'AVProductsEnabled': np.float16,
    'CountryIdentifier': np.int16,
    'CityIdentifier': np.float32,
    'OrganizationIdentifier': np.float16,
    'GeoNameIdentifier': np.float16,
    'LocaleEnglishNameIdentifier': np.int8,
    'Processor': np.int8,
    'OsBuild': np.int16,
    'OsSuite': np.int16,
    'OsPlatformSubRelease': np.int8,
    'OsBuildLab': np.int16,
    'SkuEdition': np.int8,
    'IsProtected': np.float16,
    'IeVerIdentifier': np.float16,
    'SmartScreen': np.int8,
    'Firewall': np.float16,
    'Census_MDC2FormFactor': np.int8,
    'Census_OEMNameIdentifier': np.float16,
    'Census_OEMModelIdentifier': np.float32,
    'Census_ProcessorCoreCount': np.float16,
    'Census_ProcessorModelIdentifier': np.float16,
    'Census_PrimaryDiskTotalCapacity': np.float32,
    'Census_PrimaryDiskTypeName': np.int8,
    'Census_SystemVolumeTotalCapacity': np.float32,
    'Census_HasOpticalDiskDrive': np.int8,
    'Census_TotalPhysicalRAM': np.float32,
    'Census_ChassisTypeName': np.int8,
    'Census_InternalPrimaryDiagonalDisplaySizeInInches': np.float16,
    'Census_InternalPrimaryDisplayResolutionHorizontal': np.float16,
    'Census_InternalPrimaryDisplayResolutionVertical': np.float16,
    'Census_PowerPlatformRoleName': np.int8,
    'Census_InternalBatteryNumberOfCharges': np.float32,
    'Census_OSVersion': np.float64,
    'Census_OSBranch': np.int8,
    'Census_OSBuildRevision': np.int32,
    'Census_OSEdition': np.int8,
    'Census_OSSkuName': np.int8,
    'Census_OSInstallTypeName': np.int8,
    'Census_OSInstallLanguageIdentifier': np.float16,
    'Census_OSUILocaleIdentifier': np.int16,
    'Census_OSWUAutoUpdateOptionsName': np.int8,
    'Census_GenuineStateName': np.int8,
    'Census_ActivationChannel': np.int8,
    'Census_FlightRing': np.int8,
    'Census_FirmwareManufacturerIdentifier': np.float16,
    'Census_FirmwareVersionIdentifier': np.float32,
    'Census_IsSecureBootEnabled': np.int8,
    'Census_IsVirtualDevice': np.float16,
    'Wdft_IsGamer': np.float16,
    'Wdft_RegionIdentifier': np.float16,
    'HasDetections': np.int8,
    'primary_drive_c_ratio': np.float16,
    'non_primary_drive_MB': np.float32,
    'aspect_ratio': np.float16,
    'dpi': np.float16,
    'Screen_Area': np.float64,
    'AVProductsInstalled_slim': np.float16,
    'magic_4': np.int8
}


# Load the file quickly without dtype constraints
train = pd.read_csv(r'kaggle_encoded_train.csv')

# Then cast types only for the columns that exist
for col, dtype in dtypes.items():
    if col in train.columns:
        try:
            train[col] = train[col].astype(dtype)
        except Exception as e:
            print(f"Could not cast column {col} to {dtype}: {e}")



In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder

# Assuming train is already defined

columns_to_drop = ['MachineIdentifier']
columns_to_freq_encode = ['AppVersion','AvSigVersion','Census_OSVersion','EngineVersion','OsBuildLab']
columns_for_LE = ['ProductName','RtpStateBitfield','Platform','Processor','OsVer','OsBuild','OsSuite',
                   'OsPlatformSubRelease','OsBuildLab','SkuEdition','PuaMode','SmartScreen','Census_MDC2FormFactor',
                  'Census_DeviceFamily','Census_ProcessorClass','Census_PrimaryDiskTypeName','Census_ChassisTypeName',
                  'Census_PowerPlatformRoleName','Census_InternalBatteryType','Census_OSArchitecture',
                  'Census_OSBranch','Census_OSEdition','Census_OSSkuName','Census_OSInstallTypeName',
                  'Census_OSWUAutoUpdateOptionsName','Census_GenuineStateName','Census_ActivationChannel',
                  'Census_FlightRing']
columns_with_strings = ['ProductName','Platform','Processor','OsPlatformSubRelease','OsBuildLab','SkuEdition',
                        'PuaMode','SmartScreen','Census_MDC2FormFactor','Census_DeviceFamily','Census_PrimaryDiskTypeName',
                        'Census_ChassisTypeName','Census_PowerPlatformRoleName','Census_InternalBatteryType',
                        'Census_OSArchitecture','Census_OSBranch','Census_OSEdition','Census_OSSkuName','Census_OSInstallTypeName',
                        'Census_OSWUAutoUpdateOptionsName','Census_GenuineStateName','Census_ActivationChannel',
                        'Census_FlightRing']

# Drop unwanted columns
def frequency_encoding(df, feature):
    t = df[feature].value_counts().reset_index()
    t = t.reset_index()
    t.loc[t[feature] == 1, 'level_0'] = np.nan
    t.set_index('index', inplace=True)
    max_label = t['level_0'].max() + 1
    t.fillna(max_label, inplace=True)
    return t.to_dict()['level_0']

for col in columns_to_drop:
    train.drop(columns=[col], inplace=True)

# Convert string columns to lowercase

# Use category_encoders' CountEncoder for frequency encoding
encoder = ce.CountEncoder(cols=columns_to_freq_encode)

# Apply frequency encoding
train[columns_to_freq_encode] = encoder.fit_transform(train[columns_to_freq_encode])

# Save the encoder for later use on the test set
with open('freq_encoder_kaggle.pkl', 'wb') as f:
    pickle.dump(encoder, f)

# Replace NaNs in other columns
for col in train.columns:
    if train[col].dtype == 'object':
        train[col] = train[col].fillna('NA')
    elif train[col].dtype == 'category':
        train[col] = train[col].cat.add_categories('NA').fillna('NA')
    else:
        train[col] = train[col].fillna(-1)

# Label Encoding for specific columns
label_encoders = {}
for col in columns_for_LE:
    if col not in train.columns:
        continue
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    label_encoders[col] = le

# Save the label encoders for later use
with open('label_encoders_kaggle.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)

# Drop a specific row
train.drop(index=5244810, inplace=True)

# Convert float64 and object columns to float32
for col in train.columns:
    if train[col].dtype == 'float64':
        train[col] = train[col].astype('float32')
    elif train[col].dtype == 'object':
        train[col] = train[col].astype('float32')

# Prepare the target and features
y = train['HasDetections']
X = train.drop(columns=['HasDetections'])

# Split data into train and test sets



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale the features

import numpy as np

# Replace inf/-inf with NaN
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop rows with NaN and keep track of the remaining indices
valid_idx_train = X_train.dropna().index

# Filter both X_train and y_train
X_train = X_train.loc[valid_idx_train]
y_train = y_train.loc[valid_idx_train]

X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
valid_idx_test = X_test.dropna().index
X_test = X_test.loc[valid_idx_test]
y_test = y_test.loc[valid_idx_test]
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save the scaler for later use
with open('scaler_kaggle.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [ ]:
train_data = pd.DataFrame(X_train, columns=X.columns)
train_data['HasDetections'] = y_train.values

test_data = pd.DataFrame(X_test, columns=X.columns)
test_data['HasDetections'] = y_test.values

import pandas as pd
import numpy as np

def optimize_dataframe(df):
    for col in df.columns:
        col_data = df[col]
        
        # Skip non-numeric columns
        if not np.issubdtype(col_data.dtype, np.number):
            continue

        # Check if all values are integers (i.e., no decimal part)
        if np.all(col_data.dropna() == col_data.dropna().astype(int)):
            df[col] = pd.to_numeric(col_data, downcast="integer")
        else:
            df[col] = pd.to_numeric(col_data, downcast="float")
    
    return df

# Apply optimization
train_data = optimize_dataframe(train_data)
test_data = optimize_dataframe(test_data)

# Check memory usage before and after
print(f"Memory usage after optimization: {train_data.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"Memory usage after optimization: {test_data.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:

train_data.to_csv('train_data_kaggle_processed.csv', index=False)
test_data.to_csv('validation_data_kaggle_processed.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np

# Define dtype mappings
train_dtypes = {
    'EngineVersion': np.float32, 'AppVersion': np.float32, 'AvSigVersion': np.float32, 
    'RtpStateBitfield': np.float32, 'IsSxsPassiveMode': np.float32, 
    'DefaultBrowsersIdentifier': np.float32, 'AVProductStatesIdentifier': np.float32, 
    'AVProductsInstalled': np.float32, 'AVProductsEnabled': np.float32, 
    'CountryIdentifier': np.float32, 'CityIdentifier': np.float32, 
    'OrganizationIdentifier': np.float32, 'GeoNameIdentifier': np.float32, 
    'LocaleEnglishNameIdentifier': np.float32, 'Processor': np.float32, 
    'OsBuild': np.float32, 'OsSuite': np.float32, 'OsPlatformSubRelease': np.float32, 
    'OsBuildLab': np.float32, 'SkuEdition': np.float32, 'IsProtected': np.float32, 
    'IeVerIdentifier': np.float32, 'SmartScreen': np.float32, 'Firewall': np.float32, 
    'Census_MDC2FormFactor': np.float32, 'Census_OEMNameIdentifier': np.float32, 
    'Census_OEMModelIdentifier': np.float32, 'Census_ProcessorCoreCount': np.float32, 
    'Census_ProcessorModelIdentifier': np.float32, 'Census_PrimaryDiskTotalCapacity': np.float32, 
    'Census_PrimaryDiskTypeName': np.float32, 'Census_SystemVolumeTotalCapacity': np.float32, 
    'Census_HasOpticalDiskDrive': np.float32, 'Census_TotalPhysicalRAM': np.float32, 
    'Census_ChassisTypeName': np.float32, 'Census_InternalPrimaryDiagonalDisplaySizeInInches': np.float32, 
    'Census_InternalPrimaryDisplayResolutionHorizontal': np.float32, 
    'Census_InternalPrimaryDisplayResolutionVertical': np.float32, 
    'Census_PowerPlatformRoleName': np.float32, 'Census_InternalBatteryNumberOfCharges': np.float32, 
    'Census_OSVersion': np.float32, 'Census_OSBranch': np.float32, 'Census_OSBuildRevision': np.float32, 
    'Census_OSEdition': np.float32, 'Census_OSSkuName': np.float32, 'Census_OSInstallTypeName': np.float32, 
    'Census_OSInstallLanguageIdentifier': np.float32, 'Census_OSUILocaleIdentifier': np.float32, 
    'Census_OSWUAutoUpdateOptionsName': np.float32, 'Census_GenuineStateName': np.float32, 
    'Census_ActivationChannel': np.float32, 'Census_FlightRing': np.float32, 
    'Census_FirmwareManufacturerIdentifier': np.float32, 'Census_FirmwareVersionIdentifier': np.float32, 
    'Census_IsSecureBootEnabled': np.float32, 'Census_IsVirtualDevice': np.float32, 
    'Wdft_IsGamer': np.float32, 'Wdft_RegionIdentifier': np.float32, 
    'primary_drive_c_ratio': np.float32, 'non_primary_drive_MB': np.float32, 
    'aspect_ratio': np.float32, 'dpi': np.float32, 'Screen_Area': np.float32, 
    'AVProductsInstalled_slim': np.float32, 'magic_4': np.float32, 
    'HasDetections': np.int8
}

validate_dtypes = {
    'EngineVersion': np.float32, 'AppVersion': np.float32, 'AvSigVersion': np.float32, 
    'RtpStateBitfield': np.float32, 'IsSxsPassiveMode': np.float32, 
    'DefaultBrowsersIdentifier': np.float32, 'AVProductStatesIdentifier': np.float32, 
    'AVProductsInstalled': np.float32, 'AVProductsEnabled': np.float32, 
    'CountryIdentifier': np.float32, 'CityIdentifier': np.float32, 
    'OrganizationIdentifier': np.float32, 'GeoNameIdentifier': np.float32, 
    'LocaleEnglishNameIdentifier': np.float32, 'Processor': np.float32, 
    'OsBuild': np.float32, 'OsSuite': np.float32, 'OsPlatformSubRelease': np.float32, 
    'OsBuildLab': np.float32, 'SkuEdition': np.float32, 'IsProtected': np.float32, 
    'IeVerIdentifier': np.float32, 'SmartScreen': np.float32, 'Firewall': np.float32, 
    'Census_MDC2FormFactor': np.float32, 'Census_OEMNameIdentifier': np.float32, 
    'Census_OEMModelIdentifier': np.float32, 'Census_ProcessorCoreCount': np.float32, 
    'Census_ProcessorModelIdentifier': np.float32, 'Census_PrimaryDiskTotalCapacity': np.float32, 
    'Census_PrimaryDiskTypeName': np.float32, 'Census_SystemVolumeTotalCapacity': np.float32, 
    'Census_HasOpticalDiskDrive': np.float32, 'Census_TotalPhysicalRAM': np.float32, 
    'Census_ChassisTypeName': np.float32, 'Census_InternalPrimaryDiagonalDisplaySizeInInches': np.float32, 
    'Census_InternalPrimaryDisplayResolutionHorizontal': np.float32, 
    'Census_InternalPrimaryDisplayResolutionVertical': np.float32, 
    'Census_PowerPlatformRoleName': np.float32, 'Census_InternalBatteryNumberOfCharges': np.float32, 
    'Census_OSVersion': np.float32, 'Census_OSBranch': np.float32, 'Census_OSBuildRevision': np.float32, 
    'Census_OSEdition': np.float32, 'Census_OSSkuName': np.float32, 'Census_OSInstallTypeName': np.float32, 
    'Census_OSInstallLanguageIdentifier': np.float32, 'Census_OSUILocaleIdentifier': np.float32, 
    'Census_OSWUAutoUpdateOptionsName': np.float32, 'Census_GenuineStateName': np.float32, 
    'Census_ActivationChannel': np.float32, 'Census_FlightRing': np.float32, 
    'Census_FirmwareManufacturerIdentifier': np.float32, 'Census_FirmwareVersionIdentifier': np.float32, 
    'Census_IsSecureBootEnabled': np.float32, 'Census_IsVirtualDevice': np.float32, 
    'Wdft_IsGamer': np.float32, 'Wdft_RegionIdentifier': np.float32, 
    'primary_drive_c_ratio': np.float32, 'non_primary_drive_MB': np.float32, 
    'aspect_ratio': np.float32, 'dpi': np.float32, 'Screen_Area': np.float32, 
    'AVProductsInstalled_slim': np.float32, 'magic_4': np.float32, 
    'HasDetections': np.int8
}

train_data = pd.read_csv(r'C:\Users\jithi\OneDrive\Desktop\ML project\Processed Datasets\kaggle_encoded_train.csv', dtype=train_dtypes)
validate_data = pd.read_csv(r'C:\Users\jithi\OneDrive\Desktop\ML project\Processed Datasets\kaggle_encoded_test.csv', dtype=validate_dtypes)


In [15]:
for col in ['EngineVersion','Census_OSVersion','AppVersion','AvSigVersion',
            ]:
    train[col]=train[col]-train[col][0:500000].median()


In [26]:
def encode_categorical_columns_test(x_test, columns, category_map):
    from tqdm import tqdm
    import pandas as pd

    for col in tqdm(columns):
        if col in ['MachineIdentifier'] or col in toDrop:
            continue
        x_test[col] = x_test[col].map(category_map.get(col, {}))
    return x_test


In [ ]:
test_dir=r'C:\Users\jithi\OneDrive\Desktop\ML project\microsoft-malware-prediction\test.csv'
test = pd.read_csv(test_dir, dtype=dtypes)
test = reduce_mem_usage(test)
test = test.drop(toDrop, axis=1)
test = add_features(test)
test= encode_categorical_columns_test(test,  categorical_columns,category_map  )
test = reduce_mem_usage(test)
test['SmartScreen'].replace({ '0':'off', '00000000':'off','BLOCK':'Block', 'off':'Off', 
                              'on':'On', 'requireadmin' : 'RequireAdmin', 'Enabled' : 'on', 
                              'OFF' : 'Off', 'Promt' : 'Prompt', 'prompt' : 'Prompt', 'on' : 'On', 
                              'off' : 'Off', 'warn' : 'RequireAdmin', 'requireAdmin' : 'RequireAdmin', '&#x03;' : '&#x01;'}, inplace=True)

for col in ['EngineVersion','Census_OSVersion','AppVersion','AvSigVersion',
            ]:
    test[col]=test[col]-train[col][0:500000].median()

machine_identifier = test['MachineIdentifier']
test.drop(['MachineIdentifier'], axis=1, inplace=True)



In [30]:
#print dtypes of train and test as dicts
train_dtypes = {col: str(train[col].dtype) for col in train.columns}
test_dtypes = {col: str(test[col].dtype) for col in test.columns}
print(train_dtypes)
print(test_dtypes)

{'MachineIdentifier': 'category', 'EngineVersion': 'float64', 'AppVersion': 'float64', 'AvSigVersion': 'float64', 'RtpStateBitfield': 'float16', 'IsSxsPassiveMode': 'int8', 'DefaultBrowsersIdentifier': 'float16', 'AVProductStatesIdentifier': 'float32', 'AVProductsInstalled': 'float16', 'AVProductsEnabled': 'float16', 'CountryIdentifier': 'int16', 'CityIdentifier': 'float32', 'OrganizationIdentifier': 'float16', 'GeoNameIdentifier': 'float16', 'LocaleEnglishNameIdentifier': 'int8', 'Processor': 'int8', 'OsBuild': 'int16', 'OsSuite': 'int16', 'OsPlatformSubRelease': 'int8', 'OsBuildLab': 'int16', 'SkuEdition': 'int8', 'IsProtected': 'float16', 'IeVerIdentifier': 'float16', 'SmartScreen': 'int8', 'Firewall': 'float16', 'Census_MDC2FormFactor': 'int8', 'Census_OEMNameIdentifier': 'float16', 'Census_OEMModelIdentifier': 'float32', 'Census_ProcessorCoreCount': 'float16', 'Census_ProcessorModelIdentifier': 'float16', 'Census_PrimaryDiskTotalCapacity': 'float32', 'Census_PrimaryDiskTypeName': 